# Cherry Swap Simulation

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
import ssl
import time
import math
import plotly.express as px

In [ ]:
#disable ssl for cryptory API & virtualenv
ssl._create_default_https_context = ssl._create_unverified_context

Request Data from Compound.finance API

In [ ]:
cDaiAddress = '0xf5dce57282a584d2746faf1593d3121fcac444dc'
endTimestamp = math.floor(time.time())
startTimeStamp = endTimestamp - 6 * 30 * 24 * 60 * 60 #use the most recent 6 month's worth of data.
num_buckets = 24 * 30
requestURL = "https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=%s"% (cDaiAddress, startTimeStamp, endTimestamp, num_buckets)

response = requests.get(requestURL)

Convert Data into dataframe

In [ ]:
interestOverTime = pd.DataFrame.from_dict(response.json()['borrow_rates'])
interestOverTime.head()
interestOverTime['block_time'] = pd.to_datetime(interestOverTime['block_timestamp'],unit='s')
interestOverTime['rate_per_block'] = interestOverTime['rate'] / (4 * 60 * 24 * 365)
interestOverTime.head(20)

Plot Intrest rate over time

In [ ]:
data = [go.Scatter(
        x=interestOverTime['block_time'],
        y=interestOverTime['rate']
    )]

layout = go.Layout(
    title='Compound Interst Rate Over time',
    yaxis=dict(title='Lending Rate (%)'),
    xaxis=dict(title='Date'),
    template='plotly_white')

figure = go.Figure(data=data, layout=layout)

figure.show()

Random pools

In [ ]:
# random long and short pools
pool = pd.DataFrame(np.random.randint(0,1000,size=(10, 2)), columns=list('LS'))
# total pool
pool['Total'] = pool.L + pool.S
# randomize whether test user is long or short
pool['is_long'] = np.random.randint(0,2,size=(10, 1))
# random percentage test user holds in their pool
pool['pool_percentage'] = np.random.rand(10,1)
# resulting holding of test user
pool['position'] =  pool.pool_percentage * (pool.is_long * pool.L + (1 - pool.is_long) * pool.S)
pool['total_percentage'] = pool.position / pool.Total
pool

### The Model

* pools: $P_L^0 + P_S^0 = P_T^0$
* short pool at $t+1$: $P_S^{1, fix} = P_S^0 (1 + i_0)^n$ where $i_0$ is the interest per block at $t=0$ and $n$ is the number of blocks mined between $t=1$ and $t=0$
* total pool at $t+1$: $P_T^{1, float} = P_T^0 \Pi_k (1 + i_k)^{n_k}$
* payout short: $p_j^S = f_j^S P_S^{1, fix}$ where $f_j^S$ is the fraction participant $j$ holds in the short pool
* payout long: $p_j^L = f_j^L (P_T^{1, float} - P_S^{1, fix})$

In [ ]:
sim1 = pool.copy()
sim1['Total_float'] = pool.Total
i = 1
while interestOverTime['block_timestamp'][i] < interestOverTime['block_timestamp'][0] + 30 * 24 * 60 * 60:
    number_of_blocks = (interestOverTime['block_timestamp'][i] - interestOverTime['block_timestamp'][i-1]) / 15
    sim1['Total_float'] = sim1['Total_float'] * (1 + interestOverTime['rate_per_block'][i-1])**number_of_blocks
    i += 1
sim1['Total_fixed'] = sim1.Total * (1 + interestOverTime['rate_per_block'][0])**((interestOverTime['block_timestamp'][i] - interestOverTime['block_timestamp'][0]) / 15)
sim1['payout'] = sim1.pool_percentage * (sim1.is_long * (sim1.Total_float - (sim1.S/sim1.Total) * sim1.Total_fixed) + (1 - sim1.is_long) * (sim1.S/sim1.Total) * sim1.Total_fixed)
sim1['non_pool_float_payout'] = sim1.total_percentage * sim1.Total_float
sim1['profit'] = sim1.payout - sim1.non_pool_float_payout
sim1

In [ ]:
simulations = pool.copy()
j = 0
while interestOverTime['block_timestamp'][j] + 30 * 24 * 60 * 60 <= interestOverTime['block_timestamp'].iloc[-1]:
    sim = pool.copy()
    sim['Total_float'] = pool.Total
    i = 1
    while interestOverTime['block_timestamp'][j+i] < interestOverTime['block_timestamp'][j] + 30 * 24 * 60 * 60:
        number_of_blocks = (interestOverTime['block_timestamp'][j+i] - interestOverTime['block_timestamp'][j+i-1]) / 15
        sim['Total_float'] = sim['Total_float'] * (1 + interestOverTime['rate_per_block'][j+i-1])**number_of_blocks
        i += 1
    sim['Total_fixed'] = sim.Total * (1 + interestOverTime['rate_per_block'][j])**((interestOverTime['block_timestamp'][j+i] - interestOverTime['block_timestamp'][j]) / 15)
    sim['payout'] = sim.pool_percentage * (sim.is_long * (sim.Total_float - (sim.S/sim.Total) * sim.Total_fixed) + (1 - sim.is_long) * (sim.S/sim.Total) * sim.Total_fixed)
    sim['non_pool_float_payout'] = sim.total_percentage * sim.Total_float
    sim['profit'] = sim.payout - sim.non_pool_float_payout
    simulations[j] = sim.profit
    j += 1

In [ ]:
simulations

In [ ]:
simulations_transpose = simulations.iloc[:,7:].transpose()
simulations_transpose['start_time'] = interestOverTime.iloc[:len(simulations_transpose.iloc[:, 0]), 3]

In [ ]:
simulations_transpose

In [ ]:
data = [go.Scatter(
        x=simulations_transpose['start_time'],
        y=simulations_transpose[c]
    ) for c in simulations_transpose.columns if c != 'start_time']

layout = go.Layout(
    title='Profits compared to direct investment in cDAI',
    yaxis=dict(title='Profits'),
    xaxis=dict(title='Date'),
    template='plotly_white')

figure = go.Figure(data=data, layout=layout)

figure.show()

In [ ]:
simulations_transpose_long = simulations[simulations.is_long == 1].iloc[:,7:].transpose()
simulations_transpose_long['start_time'] = interestOverTime.iloc[:len(simulations_transpose_long.iloc[:, 0]), 3]

In [ ]:
data = [go.Scatter(
        x=simulations_transpose_long['start_time'],
        y=simulations_transpose_long[c]
    ) for c in simulations_transpose_long.columns if c != 'start_time']

layout = go.Layout(
    title='Profits compared to direct investment in cDAI - only long positions',
    yaxis=dict(title='Profits'),
    xaxis=dict(title='Date'),
    template='plotly_white')

figure = go.Figure(data=data, layout=layout)

figure.show()